In [ ]:
# Install Selenium, Chromium, and Driver
!pip install pandas

# Update system packages and install Chromium and Chromedriver
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver

# Install required Python libraries
!pip install selenium
!pip install openpyxl
!pip install webdriver-manager
!pip install beautifulsoup4

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

In [ ]:
# Set up the WebDriver
chromedriver_path = "C:/Users/leneo/Downloads/chromedriver-win64/chromedriver.exe"
PATH = Service(chromedriver_path)
driver = webdriver.Chrome(service=PATH)

# Input and output file paths
input_file_path = "E:/uni/1fifth1/s/mo3a/project/12-1/first2k Edu.csv"
output_file_path = "E:/uni/1fifth1/s/mo3a/project/12-1/Outputfirst2k Edu psut wifi.csv"

# Prepare to store results
results = []

with open(input_file_path, "r") as input_file:
    lines = input_file.readlines()
    uni_count = 0
    error_counts = 0

    for index, line in enumerate(lines):
        x=round(random.uniform(0.5, 2), 1)
        time.sleep(x)
        print("wait for:")
        print(x)
        #here i tried to put random wait, to add human type

        website = line.strip()
        if not website:
            continue

        for attempt in range(1):  #  mechanism with 1 attempt
            try:
                # Open the GDPR website
                driver.get("https://2gdpr.com/")

                # Wait for the input field and enter the website
                input_field = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.NAME, "site"))
                )
                input_field.clear()
                input_field.send_keys(website)
                #print(f"Processing website: {website} (Attempt {attempt + 1})")

                # Click the "Check" button
                check_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
                check_button.click()
                

                time.sleep(0.1)



                

                while True:
                    try:
                        # Check if the "Checking..." or "Please wait..." text is still visible
                        checking_message = WebDriverWait(driver, 2).until(
                            EC.presence_of_element_located((By.ID, "h1title"))
                        )
                        status_message = driver.find_element(By.ID, "statusscan").text
                        if "Checking" in checking_message.text or "Please wait" in status_message:
                            time.sleep(1)  # Wait a second and check again
                        else:
                            break  # Exit the loop when the message is gone
                    except:
                        break  # Exit the loop if the message is not found







                info_block = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.results-page div.info-block"))
                )
                info_html = info_block.get_attribute("outerHTML")
                soup_info = BeautifulSoup(info_html, "html.parser")
                
                #print("soup_info, is ready also")

                # Check for specific icons in the "loading" div
                loading_icon = soup_info.find("div", class_="icon", id="loading")
                if loading_icon:
                    i_element = loading_icon.find("i")
                    if i_element and "class" in i_element.attrs:
                        icon_class = i_element["class"]
                        if "icon-red" in icon_class:
                            #print("Detected 'icon-redssssssssdddddddddd' in the loading div.")
                            green_count = 0
                            red_count = 1
                            status = "Error"

                            results.append([website, status, green_count])

                            print(f"{index + 1}/{len(lines)}: {website}")
                            uni_count += 1

                            # Simulate the back button to return to the main page
                            driver.back()
                            #time.sleep(0.1)  # Wait for the page to load
                            break
                    else:
                        print("oooooooooooook")






                # Wait for the "results" div to load
                WebDriverWait(driver, 10).until(lambda d: d.find_element(By.ID, "results").get_attribute("innerHTML").strip() != "")

                # Extract the "results" div
                results_div = driver.find_element(By.ID, "results")
                results_html = results_div.get_attribute("outerHTML")

                # Parse the HTML using BeautifulSoup
                soup = BeautifulSoup(results_html, "html.parser")
                extracted_results = soup.find("div", id="results")
                
                icons = soup.find_all("i", class_=["icon-green", "icon-red"])

                if extracted_results:
                    # Determine the binary output for greens (1) and reds (0)
                    binary_output = []
                    for icon in icons[:5]:  # Only consider the first 5 checks
                        if "icon-green" in icon["class"]:
                            binary_output.append(1)
                        elif "icon-red" in icon["class"]:
                            binary_output.append(0)

                    # Pad with zeros if there are fewer than 5 icons
                    while len(binary_output) < 5:
                        binary_output.append(0)

                    # Determine result status
                    green_count = binary_output.count(1)
                    if green_count == 5:
                        status = "Accepted"
                    elif green_count + binary_output.count(0) == 5:
                        status = "Not Accepted"
                    else:
                        status = "Error"

                    # Append the result
                    results.append([website, status, green_count] + binary_output)


                    
                else:
                    print(f"No 'results' div found for {website}.")
                    results.append([website, "No Results Div Found", 0])

                print(f"{index + 1}/{len(lines)}: {website}")
                uni_count += 1

                # Simulate the back button to return to the main page
                driver.back()
                time.sleep(0.1)  # Wait for the page to load

                break  # Exit retry loop on success

            except Exception as e:
                #print(f"Error processing {website} on attempt {attempt + 1}: {e}")
                print(f"Error processing {website} on attempt {attempt + 1}")




                #if attempt == 1:  # After last attempt, log the error
                results.append([website, f"Error: Try Again Dif network", 0])
                error_counts += 1
                uni_count += 1
                #time.sleep(0.1)  # Wait before retrying
        if index % 20 == 0:
            time.sleep(8)
            #each 20 websites wait a bit, to get a way from the blocking

# Quit the WebDriver
driver.quit()

# Save results to CSV
columns = ["Website", "Acceptance Status", "Green Icons Count", "Check 1", "Check 2", "Check 3", "Check 4", "Check 5"]
results_df = pd.DataFrame(results, columns=columns)
results_df.to_csv(output_file_path, index=False)


print(f"Processed {uni_count} websites with {error_counts} errors.")
print(f"Results saved to {output_file_path}")